In [1]:
import numpy as np
np.set_printoptions(formatter={'float': lambda x: "{0:0.10f}".format(x)})

In [2]:
in_rgb = np.array([255, 255, 255])
print("Input RGB: ", in_rgb)

Input RGB:  [255 255 255]


In [3]:
# Matrix used in the SDL shader
yuv_legal_to_rgb_full_709 = np.array( [
    [1.1644, 0.0, 1.7927],
    [1.1644, -0.2132, -0.5329],
    [1.1644, 2.1124, 0.0]
])

yuv_full_to_rgb_full_709 = np.array([
    [1.0, 0.0, 1.28033],
    [1.0, -0.21482, -0.38059],
    [1.0, 2.12798, 0.0]
])

def get_yuv_matrix(kr, kb):
    # See https://en.wikipedia.org/wiki/YCbCr - RGB conversion
    kg = 1.0 - kb - kr
    return np.array([
        [kr, kg, kb],
        [-0.5*(kr / (1.0 - kb)), -0.5*(kg / (1.0-kb)), 0.5],
        [0.5, -0.5 * (kg / (1.0-kr)), -0.5 * (kb / (1.0-kr))]
    ])
# From https://www.itu.int/dms_pubrec/itu-r/rec/bt/R-REC-BT.709-6-201506-I!!PDF-E.pdf
kr_709 = 0.2126
kb_709 = 0.0722
kr_2020 = 0.2627
kb_2020 = 0.0593

In [4]:
rgb_to_yuv_709 = get_yuv_matrix(kr_709, kb_709)
print("RGB to YUV 709 matrix: \n", rgb_to_yuv_709)

# Minimum Y is 16, max 235
# Minimum UV is 16, max 240
# See https://www.itu.int/dms_pubrec/itu-r/rec/bt/R-REC-BT.709-6-201506-I!!PDF-E.pdf
range_multiplier = np.array([(235.0-16.0)/255.0, (240.0 - 16.0)/255.0, (240.0 - 16.0)/255.0])
yuv_offset_legal = np.array([16.0, 128.0, 128.0])
yuv_offset_full = np.array([0.0, 128.0, 128.0])

print(np.linalg.inv(rgb_to_yuv_709))

RGB to YUV 709 matrix: 
 [[0.2126000000 0.7152000000 0.0722000000]
 [-0.1145721061 -0.3854278939 0.5000000000]
 [0.5000000000 -0.4541529083 -0.0458470917]]
[[1.0000000000 0.0000000000 1.5748000000]
 [1.0000000000 -0.1873242729 -0.4681242729]
 [1.0000000000 1.8556000000 0.0000000000]]


In [5]:
yuv_out_legal = np.matmul(rgb_to_yuv_709, in_rgb)
yuv_out_legal = np.multiply(yuv_out_legal, range_multiplier)
yuv_out_legal = yuv_out_legal + yuv_offset_legal
print("YUV out legal is: ", yuv_out_legal)

YUV out legal is:  [235.0000000000 128.0000000000 128.0000000000]


In [6]:
yuv_out_full = np.matmul(rgb_to_yuv_709, in_rgb) + yuv_offset_full
print("YUV out full is: ", yuv_out_full)

YUV out full is:  [255.0000000000 128.0000000000 128.0000000000]


In [7]:
# Control that the conversion was the identity
out_rgb_from_legal = np.matmul(yuv_legal_to_rgb_full_709, yuv_out_legal - yuv_offset_legal)
print("Resulting RGB - yuv legal: ", np.round(out_rgb_from_legal))

out_rgb_from_full = np.matmul(yuv_full_to_rgb_full_709, yuv_out_full - yuv_offset_full)
print("Resulting RGB - yuv full: ", np.round(out_rgb_from_full))

Resulting RGB - yuv legal:  [255.0000000000 255.0000000000 255.0000000000]
Resulting RGB - yuv full:  [255.0000000000 255.0000000000 255.0000000000]


In [8]:
# Value from Alex's DeckLink stream (255, 0, 0)
test_yuv = np.array([54, 98, 254])
renderer_matrix = yuv_full_to_rgb_full_709
zimg_matrix = np.linalg.inv(rgb_to_yuv_709)
print(np.matmul(renderer_matrix, test_yuv - yuv_offset_full))
print(np.matmul(zimg_matrix, test_yuv - yuv_offset_full))

[215.3215800000 12.4902600000 -9.8394000000]
[252.4248000000 0.6360697987 -1.6680000000]


In [27]:
asyuv = np.matmul(rgb_to_yuv_709, np.array([255, 255, 255])) + np.array([0, 128, 128])
print(asyuv)
print(np.linalg.inv(rgb_to_yuv_709))
rgbFull = np.matmul(np.linalg.inv(rgb_to_yuv_709), ((asyuv - yuv_offset_full)))
print(rgbFull)

[255.0000000000 128.0000000000 128.0000000000]
[[1.0000000000 0.0000000000 1.5748000000]
 [1.0000000000 -0.1873242729 -0.4681242729]
 [1.0000000000 1.8556000000 0.0000000000]]
[255.0000000000 255.0000000000 255.0000000000]


[59925.0000000000 32640.0000000000 32640.0000000000]
